# Loading the data



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the CSV files into DataFrames
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

# Display the first few rows of the DataFrames
print(train_features.head())
print(train_labels.head())

    uid    age_03     urban_03  married_03  n_mar_03    edu_gru_03  \
0  aace       NaN          NaN         NaN       NaN           NaN   
1  aanz       NaN          NaN         NaN       NaN           NaN   
2  aape       NaN          NaN         NaN       NaN           NaN   
3  aard  1. 50–59  1. 100,000+  3. Widowed       1.0  3. 7–9 years   
4  ablr       NaN          NaN         NaN       NaN           NaN   

  n_living_child_03  migration_03 glob_hlth_03  adl_dress_03  ...  \
0               NaN           NaN          NaN           NaN  ...   
1               NaN           NaN          NaN           NaN  ...   
2               NaN           NaN          NaN           NaN  ...   
3         1. 1 or 2           0.0      4. Fair           0.0  ...   
4               NaN           NaN          NaN           NaN  ...   

            rrelgimp_12            rrfcntx_m_12              rsocact_m_12  \
0  2.somewhat important                 9.Never                   9.Never   
1      1.v

The dataset consists of two main files that we will be working with:

1. `train_features.csv`: Contains the predictors based on social determinants of health (SDOH) from the Mexican Health and Aging Study (MHAS). This file has 184 columns, including:

- `uid`: Unique identifier for each individual.
- Variables across years 2003 and 2012, with prefixes like age_03, urban_03, edu_gru_03, etc., representing different characteristics (age, urban status, marital status, education level, etc.).
- Several columns appear to contain categorical data encoded with descriptors (e.g., "1. 50–59" for age, "3. Widowed" for marital status).
- Many entries are marked as NaN, suggesting missing data for some individuals and variables.

2. `train_labels.csv`: Contains the target variable for prediction.

- `uid`: Unique identifier matching those in train_features.
- `year`: Year when the cognitive composite score was recorded (either 2016 or 2021).
- `composite_score`: Cognitive composite score, representing the outcome we aim to predict.

In [ ]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Columns: 184 entries, uid to j11_12
dtypes: float64(140), object(44)
memory usage: 4.6+ MB


`train_features` contains 3276 entries with 184 columns.

In [ ]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4343 entries, 0 to 4342
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   uid              4343 non-null   object
 1   year             4343 non-null   int64 
 2   composite_score  4343 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 101.9+ KB


`train_labels` contains 4334 entries with 3 columns.


In [ ]:
# Checking if all uids in train_labels are present in train_features
missing_uids_in_features = set(train_labels['uid']) - set(train_features['uid'])

# Checking if there are duplicate uid-year pairs in train_labels which should be unique
duplicate_uid_year_pairs = train_labels.duplicated(subset=['uid', 'year']).sum()

# Results summary
missing_uids_in_features, duplicate_uid_year_pairs


(set(), 0)

Data Integrity Check Results
1. UID Consistency:

    -All uid values in train_labels are present in train_features, confirming there are no missing individuals between the two datasets.
2. Unique UID-Year Pairs:

    - No duplicate uid-year pairs in train_labels, ensuring that each UID-year combination corresponds to a unique cognitive score.

In [ ]:
# Display column names in train_features to check for consistency in naming conventions
train_features.columns.tolist()


['uid',
 'age_03',
 'urban_03',
 'married_03',
 'n_mar_03',
 'edu_gru_03',
 'n_living_child_03',
 'migration_03',
 'glob_hlth_03',
 'adl_dress_03',
 'adl_walk_03',
 'adl_bath_03',
 'adl_eat_03',
 'adl_bed_03',
 'adl_toilet_03',
 'n_adl_03',
 'iadl_money_03',
 'iadl_meds_03',
 'iadl_shop_03',
 'iadl_meals_03',
 'n_iadl_03',
 'depressed_03',
 'hard_03',
 'restless_03',
 'happy_03',
 'lonely_03',
 'enjoy_03',
 'sad_03',
 'tired_03',
 'energetic_03',
 'n_depr_03',
 'cesd_depressed_03',
 'hypertension_03',
 'diabetes_03',
 'resp_ill_03',
 'arthritis_03',
 'hrt_attack_03',
 'stroke_03',
 'cancer_03',
 'n_illnesses_03',
 'bmi_03',
 'exer_3xwk_03',
 'alcohol_03',
 'tobacco_03',
 'test_chol_03',
 'test_tuber_03',
 'test_diab_03',
 'test_pres_03',
 'hosp_03',
 'visit_med_03',
 'out_proc_03',
 'visit_dental_03',
 'imss_03',
 'issste_03',
 'pem_def_mar_03',
 'insur_private_03',
 'insur_other_03',
 'insured_03',
 'decis_famil_03',
 'decis_personal_03',
 'employment_03',
 'age_12',
 'urban_12',
 'ma

Column Naming Convention Observations
1. Year-Specific Suffixes:

    - Most columns follow a convention where suffixes _03 and _12 indicate data collected in 2003 and 2012, respectively.
    - Exceptions are columns like ragender, rameduc_m, and rafeduc_m (gender and parental education), which do not have year-specific suffixes, likely indicating they are static variables.
2. Underscore Separations:

    - Column names generally use underscores (_) for separating words, making them consistent and readable. However, some columns use combined letters without separation (e.g., iadl_money_03 and adl_dress_03), which is still consistent across similar variables.
3. Mixed Naming for Health and Social Determinants:

    - Variables related to health coverage (e.g., imss_03, issste_03) and household income sources (e.g., hincome_03, hinc_business_03) are consistently named with abbreviations followed by underscores and year suffixes.
    - Social activities and lifestyle variables, such as vax_flu_12, attends_club_12, and rrfcntx_m_12, follow similar conventions, maintaining readability and consistency.

In [ ]:
# Checking the proportion of missing values per column
missing_values_features = train_features.isnull().mean() * 100  # percentage of missing values in train_features
missing_values_labels = train_labels.isnull().mean() * 100      # percentage of missing values in train_labels

print(missing_values_features)
print(missing_values_labels)

uid            0.000000
age_03        31.623932
urban_03      31.562882
married_03    31.562882
n_mar_03      32.173382
                ...    
a21_12        98.717949
a22_12        98.901099
a33b_12       98.717949
a34_12        35.531136
j11_12         2.289377
Length: 184, dtype: float64
uid                0.0
year               0.0
composite_score    0.0
dtype: float64


Missing values are prevalent across both years (2003 and 2012), with some variables (like a21_12 and a22_12) having over 98% missing values.

High proportions of missing data suggest selective survey questions or responses

In [ ]:
# Separating categorical and numerical columns in train_features
categorical_columns = train_features.select_dtypes(include=['object']).columns.tolist()
numerical_columns = train_features.select_dtypes(include=['float64', 'int64']).columns.tolist()

print(categorical_columns)
print(numerical_columns)

print(f"Number of categorical columns: {len(categorical_columns)}")
print(f"Number of numerical columns: {len(numerical_columns)}")


['uid', 'age_03', 'urban_03', 'married_03', 'edu_gru_03', 'n_living_child_03', 'glob_hlth_03', 'bmi_03', 'decis_famil_03', 'employment_03', 'age_12', 'urban_12', 'married_12', 'edu_gru_12', 'n_living_child_12', 'glob_hlth_12', 'bmi_12', 'decis_famil_12', 'decis_personal_12', 'employment_12', 'satis_ideal_12', 'satis_excel_12', 'satis_fine_12', 'cosas_imp_12', 'wouldnt_change_12', 'memory_12', 'ragender', 'rameduc_m', 'rafeduc_m', 'sgender_03', 'rjlocc_m_03', 'rjobend_reason_03', 'rrelgimp_03', 'sgender_12', 'rjlocc_m_12', 'rjobend_reason_12', 'rrelgimp_12', 'rrfcntx_m_12', 'rsocact_m_12', 'rrelgwk_12', 'a22_12', 'a33b_12', 'a34_12', 'j11_12']
['n_mar_03', 'migration_03', 'adl_dress_03', 'adl_walk_03', 'adl_bath_03', 'adl_eat_03', 'adl_bed_03', 'adl_toilet_03', 'n_adl_03', 'iadl_money_03', 'iadl_meds_03', 'iadl_shop_03', 'iadl_meals_03', 'n_iadl_03', 'depressed_03', 'hard_03', 'restless_03', 'happy_03', 'lonely_03', 'enjoy_03', 'sad_03', 'tired_03', 'energetic_03', 'n_depr_03', 'cesd_de

In [ ]:
# Display unique values for each categorical column in train_features to understand the data entries better
unique_values_categorical = {col: train_features[col].unique() for col in categorical_columns}

unique_values_categorical

{'uid': array(['aace', 'aanz', 'aape', ..., 'zzci', 'zzjb', 'zzti'], dtype=object),
 'age_03': array([nan, '1. 50–59', '3. 70–79', '2. 60–69', '0. 49 or younger',
        '4. 80+'], dtype=object),
 'urban_03': array([nan, '1. 100,000+', '0. <100,000'], dtype=object),
 'married_03': array([nan, '3. Widowed', '1. Married or in civil union', '4. Single',
        '2. Separated or divorced'], dtype=object),
 'edu_gru_03': array([nan, '3. 7–9 years', '1. 1–5 years', '0. No education',
        '2. 6 years', '4. 10+ years'], dtype=object),
 'n_living_child_03': array([nan, '1. 1 or 2', '3. 5 or 6', '0. No children', '2. 3 or 4',
        '4. 7+'], dtype=object),
 'glob_hlth_03': array([nan, '4. Fair', '5. Poor', '3. Good', '1. Excellent',
        '2. Very good'], dtype=object),
 'bmi_03': array([nan, '3. Overweight', '4. Obese', '1. Underweight',
        '2. Normal weight', '5. Morbidly obese'], dtype=object),
 'decis_famil_03': array([nan, '2. Approximately equal weight', '1. Respondent', '3. 

In [ ]:
# Display unique values for each numerical column in train_features to understand the data entries better
unique_values_numerical = {col: train_features[col].unique() for col in numerical_columns}

unique_values_numerical

{'n_mar_03': array([nan,  1.,  2.,  3.,  0.,  4.,  5.]),
 'migration_03': array([nan,  0.,  1.]),
 'adl_dress_03': array([nan,  0.,  1.]),
 'adl_walk_03': array([nan,  0.,  1.]),
 'adl_bath_03': array([nan,  0.,  1.]),
 'adl_eat_03': array([nan,  0.,  1.]),
 'adl_bed_03': array([nan,  0.,  1.]),
 'adl_toilet_03': array([nan,  0.,  1.]),
 'n_adl_03': array([nan,  0.,  1.,  2.,  3.,  5.,  4.]),
 'iadl_money_03': array([nan,  0.,  1.]),
 'iadl_meds_03': array([nan,  0.,  1.]),
 'iadl_shop_03': array([nan,  0.,  1.]),
 'iadl_meals_03': array([nan,  0.,  1.]),
 'n_iadl_03': array([nan,  0.,  1.,  3.,  2.,  4.]),
 'depressed_03': array([nan,  1.,  0.]),
 'hard_03': array([nan,  0.,  1.]),
 'restless_03': array([nan,  1.,  0.]),
 'happy_03': array([nan,  1.,  0.]),
 'lonely_03': array([nan,  1.,  0.]),
 'enjoy_03': array([nan,  1.,  0.]),
 'sad_03': array([nan,  1.,  0.]),
 'tired_03': array([nan,  1.,  0.]),
 'energetic_03': array([nan,  0.,  1.]),
 'n_depr_03': array([nan,  6.,  7.,  3.,  5

In [ ]:
# Identify columns with missing values in train_features and calculate the percentage of missing values for each
missing_values_features = train_features.isnull().mean() * 100
columns_with_missing_values = missing_values_features[missing_values_features > 0]

# Extract the columns with missing values for inspection
missing_values_data = train_features[columns_with_missing_values.index]

# Display the percentage of missing values in each column and the unique values for each
missing_values_info = {col: {"missing_percentage": columns_with_missing_values[col],
                             "unique_values": missing_values_data[col].unique()}
                       for col in columns_with_missing_values.index}

missing_values_info


{'age_03': {'missing_percentage': 31.62393162393162,
  'unique_values': array([nan, '1. 50–59', '3. 70–79', '2. 60–69', '0. 49 or younger',
         '4. 80+'], dtype=object)},
 'urban_03': {'missing_percentage': 31.56288156288156,
  'unique_values': array([nan, '1. 100,000+', '0. <100,000'], dtype=object)},
 'married_03': {'missing_percentage': 31.56288156288156,
  'unique_values': array([nan, '3. Widowed', '1. Married or in civil union', '4. Single',
         '2. Separated or divorced'], dtype=object)},
 'n_mar_03': {'missing_percentage': 32.17338217338217,
  'unique_values': array([nan,  1.,  2.,  3.,  0.,  4.,  5.])},
 'edu_gru_03': {'missing_percentage': 31.868131868131865,
  'unique_values': array([nan, '3. 7–9 years', '1. 1–5 years', '0. No education',
         '2. 6 years', '4. 10+ years'], dtype=object)},
 'n_living_child_03': {'missing_percentage': 31.929181929181926,
  'unique_values': array([nan, '1. 1 or 2', '3. 5 or 6', '0. No children', '2. 3 or 4',
         '4. 7+'], dty

- Total Columns with Missing Values: 197 out of all the columns in train_features.csv.
- Range of Missing Percentages: From as low as ~2.29% to as high as ~99.27%.
- Categories of Missingness:
    - Low Missingness (<10%): Generally recent data from 2012 (_12 variables).
    - Moderate Missingness (10%-50%): Mix of 2003 (_03) and 2012 (_12) variables.
    - High Missingness (>50%): Mostly variables from 2003 (_03), especially occupation-related data.


To prepare for different strategies in handling missing data, we'll separate the variables into four categories based on their percentage of missing values:

1. Low Missingness: Variables with missing values less than or equal to 10%.
2. Moderate Missingness: Variables with missing values greater than 10% and less than or equal to 50%.
3. High Missingness: Variables with missing values greater than 50%, and less than 90%
4. Very High Missingness: Variables with missing values greater than 90%

In [ ]:
# Define thresholds for missingness categories
low_threshold = 10       # 10% or less is considered low missingness
moderate_threshold = 50  # Between 10% and 50% is moderate missingness
high_threshold = 90      # Between 50% and 90% is high missingness

# Separate columns based on missingness percentages
low_missingness_columns = missing_values_features[missing_values_features <= low_threshold].index.tolist()
moderate_missingness_columns = missing_values_features[
    (missing_values_features > low_threshold) & (missing_values_features <= moderate_threshold)
].index.tolist()
high_missingness_columns = missing_values_features[
    (missing_values_features > moderate_threshold) & (missing_values_features <= high_threshold)
].index.tolist()
very_high_missingness_columns = missing_values_features[missing_values_features > high_threshold].index.tolist()

# Print the number of variables in each category
print(f"Low missingness columns ({len(low_missingness_columns)}): {low_missingness_columns}\n")
print(f"Moderate missingness columns ({len(moderate_missingness_columns)}): {moderate_missingness_columns}\n")
print(f"High missingness columns ({len(high_missingness_columns)}): {high_missingness_columns}\n")
print(f"Very high missingness columns ({len(very_high_missingness_columns)}): {very_high_missingness_columns}\n")

Low missingness columns (93): ['uid', 'age_12', 'urban_12', 'married_12', 'n_mar_12', 'edu_gru_12', 'n_living_child_12', 'migration_12', 'glob_hlth_12', 'adl_dress_12', 'adl_walk_12', 'adl_bath_12', 'adl_eat_12', 'adl_bed_12', 'adl_toilet_12', 'n_adl_12', 'iadl_money_12', 'iadl_meds_12', 'iadl_shop_12', 'iadl_meals_12', 'n_iadl_12', 'depressed_12', 'hard_12', 'restless_12', 'happy_12', 'lonely_12', 'enjoy_12', 'sad_12', 'tired_12', 'energetic_12', 'n_depr_12', 'cesd_depressed_12', 'hypertension_12', 'diabetes_12', 'resp_ill_12', 'arthritis_12', 'hrt_attack_12', 'stroke_12', 'cancer_12', 'n_illnesses_12', 'exer_3xwk_12', 'alcohol_12', 'tobacco_12', 'test_chol_12', 'test_tuber_12', 'test_diab_12', 'test_pres_12', 'hosp_12', 'visit_med_12', 'out_proc_12', 'visit_dental_12', 'imss_12', 'issste_12', 'pem_def_mar_12', 'insur_private_12', 'insur_other_12', 'insured_12', 'decis_personal_12', 'employment_12', 'vax_flu_12', 'vax_pneu_12', 'seg_pop_12', 'care_adult_12', 'care_child_12', 'voluntee

For low missingness columns, we can use simple imputation methods like mean (for numerical variables) or mode (for categorical variables).

In [ ]:
# Get the data types of the columns
column_types = train_features.dtypes

# Identify numerical and categorical columns
numerical_columns = column_types[column_types != 'object'].index.tolist()
categorical_columns = column_types[column_types == 'object'].index.tolist()

# Now, among the low missingness columns, separate numerical and categorical
low_missing_numerical = [col for col in low_missingness_columns if col in numerical_columns]
low_missing_categorical = [col for col in low_missingness_columns if col in categorical_columns]

# Impute numerical columns with mean
for col in low_missing_numerical:
    mean_value = train_features[col].mean()
    train_features[col].fillna(mean_value, inplace=True)

# Impute categorical columns with mode
for col in low_missing_categorical:
    mode_value = train_features[col].mode()
    if not mode_value.empty:
        mode_value = mode_value[0]
        train_features[col].fillna(mode_value, inplace=True)
    else:
        # If mode is empty (all values are NaN), fill with a placeholder or keep NaN
        train_features[col].fillna('Unknown', inplace=True)

# Verify that there are no missing values in low missingness columns
missing_after_imputation = train_features[low_missingness_columns].isnull().sum()
print("Missing values after imputation in low missingness columns:")
print(missing_after_imputation)


Missing values after imputation in low missingness columns:
uid             0
age_12          0
urban_12        0
married_12      0
n_mar_12        0
               ..
rrelgimp_12     0
rrfcntx_m_12    0
rsocact_m_12    0
rrelgwk_12      0
j11_12          0
Length: 93, dtype: int64


For moderate misssingness columns, for categorical variables, we can introduce a new category 'Missing' and for numerical variables, we can create a missingness indicator and impute missing values with the median

In [ ]:
# Separate numerical and categorical columns among moderate missingness columns
moderate_missing_numerical = [col for col in moderate_missingness_columns if column_types[col] != 'object']
moderate_missing_categorical = [col for col in moderate_missingness_columns if column_types[col] == 'object']

# For categorical variables, replace missing values with a new category 'Missing'
for col in moderate_missing_categorical:
    train_features[col] = train_features[col].fillna('Missing')

# For numerical variables, we'll create a missingness indicator and impute missing values with the median

for col in moderate_missing_numerical:
    # Create missingness indicator
    train_features[f'{col}_missing'] = train_features[col].isnull().astype(int)
    # Impute missing values with the median
    median_value = train_features[col].median()
    train_features[col].fillna(median_value, inplace=True)

# Verify that missing values have been handled in moderate missingness columns
missing_after_imputation = train_features[moderate_missingness_columns].isnull().sum()
print("Missing values after handling in moderate missingness columns:")
print(missing_after_imputation)

Missing values after handling in moderate missingness columns:
age_03             0
urban_03           0
married_03         0
n_mar_03           0
edu_gru_03         0
                  ..
sgender_12         0
rjlocc_m_12        0
searnings_12       0
sinc_pension_12    0
a34_12             0
Length: 78, dtype: int64


Potential Issues:

1. Multicollinearity:

- Including both the original variable (with imputed values) and the missingness indicator could introduce multicollinearity.
- Consider assessing variance inflation factors (VIF) if multicollinearity is a concern.
2. Model Compatibility:

- Some machine learning algorithms can handle missing values internally (e.g., decision trees in certain implementations). However, creating missingness indicators and imputing ensures compatibility across different models.


Since columns with high (>50%) and very high (>90%) missingness contain a significant proportion of missing data, it's reasonable to consider dropping them out of our dataset.

In [ ]:
# Combine high and very high missingness columns
columns_to_drop = high_missingness_columns + very_high_missingness_columns

# Drop these columns from the DataFrame
train_features_dropped = train_features.drop(columns=columns_to_drop)

# Print the number of columns before and after dropping
print(f"Number of columns before dropping: {train_features.shape[1]}")
print(f"Number of columns after dropping: {train_features_dropped.shape[1]}")


Number of columns before dropping: 246
Number of columns after dropping: 233


Next we deal with categorical variables. We need to identify ordinal and nominal variables and encode them appropriately.

In [ ]:
# Get data types of all columns
data_types = train_features_dropped.dtypes

# Identify categorical variables (object type or categorical type)
categorical_columns = data_types[data_types == 'object'].index.tolist()

print(f"Categorical variables ({len(categorical_columns)}): {categorical_columns}")


Categorical variables (37): ['uid', 'age_03', 'urban_03', 'married_03', 'edu_gru_03', 'n_living_child_03', 'glob_hlth_03', 'employment_03', 'age_12', 'urban_12', 'married_12', 'edu_gru_12', 'n_living_child_12', 'glob_hlth_12', 'bmi_12', 'decis_famil_12', 'decis_personal_12', 'employment_12', 'satis_ideal_12', 'satis_excel_12', 'satis_fine_12', 'cosas_imp_12', 'wouldnt_change_12', 'memory_12', 'ragender', 'rameduc_m', 'rafeduc_m', 'sgender_03', 'rrelgimp_03', 'sgender_12', 'rjlocc_m_12', 'rrelgimp_12', 'rrfcntx_m_12', 'rsocact_m_12', 'rrelgwk_12', 'a34_12', 'j11_12']


In [ ]:
# Identify ordinal variables based on data descriptions
ordinal_variables = [
    'age_03',              # Binned age group
    'edu_gru_03',          # Binned education level
    'n_living_child_03',   # Binned number of living children
    'glob_hlth_03',        # Self-reported global health
    'age_12',              # Binned age group
    'edu_gru_12',          # Binned education level
    'n_living_child_12',   # Binned number of living children
    'glob_hlth_12',        # Self-reported global health
    'bmi_12',              # Binned body mass index
    'decis_famil_12',      # Weight in family decisions
    'decis_personal_12',   # Weight over personal decisions
    'satis_ideal_12',      # Agreement with life satisfaction statements
    'satis_excel_12',
    'satis_fine_12',
    'cosas_imp_12',
    'wouldnt_change_12',
    'memory_12',           # Self-reported memory
    'rameduc_m',           # Mother's education level
    'rafeduc_m',           # Father's education level
    'rrelgimp_03',         # Importance of religion
    'rrelgimp_12',
    'rrfcntx_m_12',        # Frequency of seeing friends/relatives
    'rsocact_m_12',        # Frequency of social activities
    'rrelgwk_12',          # Participation in weekly religious services
    'a34_12'               # English proficiency
]

# Nominal variables are the rest of the categorical variables
nominal_variables = [col for col in categorical_columns if col not in ordinal_variables]

# Output the lists
print(f"Ordinal variables ({len(ordinal_variables)}): {ordinal_variables}\n")
print(f"Nominal variables ({len(nominal_variables)}): {nominal_variables}\n")

Ordinal variables (25): ['age_03', 'edu_gru_03', 'n_living_child_03', 'glob_hlth_03', 'age_12', 'edu_gru_12', 'n_living_child_12', 'glob_hlth_12', 'bmi_12', 'decis_famil_12', 'decis_personal_12', 'satis_ideal_12', 'satis_excel_12', 'satis_fine_12', 'cosas_imp_12', 'wouldnt_change_12', 'memory_12', 'rameduc_m', 'rafeduc_m', 'rrelgimp_03', 'rrelgimp_12', 'rrfcntx_m_12', 'rsocact_m_12', 'rrelgwk_12', 'a34_12']

Nominal variables (12): ['uid', 'urban_03', 'married_03', 'employment_03', 'urban_12', 'married_12', 'employment_12', 'ragender', 'sgender_03', 'sgender_12', 'rjlocc_m_12', 'j11_12']



In [ ]:
train_features_dropped['edu_gru_12'].unique()

array(['0. No education', '3. 7–9 years', '1. 1–5 years', '2. 6 years',
       '4. 10+ years'], dtype=object)

In [ ]:
# Mappings for ordinal variables

# age_03 and age_12: Binned age group
age_mapping = {
    '0. 49 or younger': 0,
    '1. 50–59': 1,
    '2. 60–69': 2,
    '3. 70–79': 3,
    '4. 80+': 4,
    'Missing': np.nan  # or choose an appropriate code for missing values
}


# edu_gru_03 and edu_gru_12: Binned education level
education_mapping = {
    '0. No education': 0,
    '1. 1–5 years': 1,
    '2. 6 years': 2,
    '3. 7–9 years': 3,
    '4. 10+ years': 4,
    'Missing': np.nan
}


# n_living_child_03 and n_living_child_12: Binned number of living children
n_living_child_mapping = {
    '0. No children': 0,
    '1. 1 or 2': 1,
    '2. 3 or 4': 2,
    '3. 5 or 6': 3,
    '4. 7+': 4,
    'Missing': np.nan
}


# glob_hlth_03 and glob_hlth_12: Self-reported global health
glob_health_mapping = {
    '1. Excellent': 5,
    '2. Very good': 4,
    '3. Good': 3,
    '4. Fair': 2,
    '5. Poor': 1,
    'Missing': np.nan
}


# bmi_12: Binned body mass index
bmi_mapping = {
    '1. Underweight': 1,
    '2. Normal weight': 2,
    '3. Overweight': 3,
    '4. Obese': 4,
    '5. Morbidly obese': 5,
    'Missing': np.nan
}


# decis_famil_12 Weight in family decisions
decis_famil_mapping = {
    '1. Respondent': 1,
    '2. Approximately equal weight': 2,
    '3. Spouse': 3,
    'Missing': np.nan
}

# decis_personal_12 Weight in personal decisions
decis_personal_mapping = {
    '1. A lot': 3,
    '2. A little': 2,
    '3. None': 1
}

# satis_ideal_12, satis_excel_12, satis_fine_12, cosas_imp_12, wouldnt_change_12:
# Agreement with life satisfaction statements
agreement_mapping = {
    '1. Agrees': 3,
    '2. Neither agrees nor disagrees': 2,
    '3. Disagrees': 1,
    'Missing': np.nan
}


# memory_12: Self-reported memory
memory_mapping = {
    '1. Excellent': 5,
    '2. Very good': 4,
    '3. Good': 3,
    '4. Fair': 2,
    '5. Poor': 1,
    'Missing': np.nan
}


# rameduc_m and rafeduc_m: Mother's and Father's education level
parent_education_mapping = {
    '1.None': 1,
    '2.Some primary': 2,
    '3.Primary': 3,
    '4.More than primary': 4,
    'Missing': np.nan
}


# rrelgimp_03 and rrelgimp_12: Importance of religion
religion_importance_mapping = {
    '1.very important': 3,
    '2.somewhat important': 2,
    '3.not important': 1,
    'Missing': np.nan
}


# rrfcntx_m_12 and rsocact_m_12: Frequency of social interactions
frequency_mapping = {
    '1.Almost every day': 9,
    '2.4 or more times a week': 8,
    '3.2 or 3 times a week': 7,
    '4.Once a week': 6,
    '5.4 or more times a month': 5,
    '6.2 or 3 times a month': 4,
    '7.Once a month': 3,
    '8.Almost Never, sporadic': 2,
    '9.Never': 1,
    'Missing': np.nan
}


# rrelgwk_12: Participation in weekly religious services
religious_services_mapping = {
    '1.Yes': 1,
    '0.No': 0,
    'Missing': np.nan
}


# 13. a34_12: English proficiency
english_proficiency_mapping = {
    'Yes 1': 1,
    'No 2': 0,
    'Missing': np.nan
}

# Compile all mappings into a dictionary for easy access
ordinal_mappings = {
    'age_03': age_mapping,
    'age_12': age_mapping,
    'edu_gru_03': education_mapping,
    'edu_gru_12': education_mapping,
    'n_living_child_03': n_living_child_mapping,
    'n_living_child_12': n_living_child_mapping,
    'glob_hlth_03': glob_health_mapping,
    'glob_hlth_12': glob_health_mapping,
    'bmi_12': bmi_mapping,
    'decis_famil_12': decis_famil_mapping,
    'decis_personal_12': decis_personal_mapping,
    'satis_ideal_12': agreement_mapping,
    'satis_excel_12': agreement_mapping,
    'satis_fine_12': agreement_mapping,
    'cosas_imp_12': agreement_mapping,
    'wouldnt_change_12': agreement_mapping,
    'memory_12': memory_mapping,
    'rameduc_m': parent_education_mapping,
    'rafeduc_m': parent_education_mapping,
    'rrelgimp_03': religion_importance_mapping,
    'rrelgimp_12': religion_importance_mapping,
    'rrfcntx_m_12': frequency_mapping,
    'rsocact_m_12': frequency_mapping,
    'rrelgwk_12': religious_services_mapping,
    'a34_12': english_proficiency_mapping
}

# Print out the mappings
for var, mapping in ordinal_mappings.items():
    print(f"Mapping for {var}:")
    for category, code in mapping.items():
        print(f"  '{category}': {code}")
    print()


Mapping for age_03:
  '0. 49 or younger': 0
  '1. 50–59': 1
  '2. 60–69': 2
  '3. 70–79': 3
  '4. 80+': 4
  'Missing': nan

Mapping for age_12:
  '0. 49 or younger': 0
  '1. 50–59': 1
  '2. 60–69': 2
  '3. 70–79': 3
  '4. 80+': 4
  'Missing': nan

Mapping for edu_gru_03:
  '0. No education': 0
  '1. 1–5 years': 1
  '2. 6 years': 2
  '3. 7–9 years': 3
  '4. 10+ years': 4
  'Missing': nan

Mapping for edu_gru_12:
  '0. No education': 0
  '1. 1–5 years': 1
  '2. 6 years': 2
  '3. 7–9 years': 3
  '4. 10+ years': 4
  'Missing': nan

Mapping for n_living_child_03:
  '0. No children': 0
  '1. 1 or 2': 1
  '2. 3 or 4': 2
  '3. 5 or 6': 3
  '4. 7+': 4
  'Missing': nan

Mapping for n_living_child_12:
  '0. No children': 0
  '1. 1 or 2': 1
  '2. 3 or 4': 2
  '3. 5 or 6': 3
  '4. 7+': 4
  'Missing': nan

Mapping for glob_hlth_03:
  '1. Excellent': 5
  '2. Very good': 4
  '3. Good': 3
  '4. Fair': 2
  '5. Poor': 1
  'Missing': nan

Mapping for glob_hlth_12:
  '1. Excellent': 5
  '2. Very good': 4
 

In [ ]:
# Check for unmapped categories in each ordinal variable
for var, mapping in ordinal_mappings.items():
    if var in train_features_dropped.columns:
        unique_values = set(train_features_dropped[var].unique())
        mapped_values = set(mapping.keys())
        unmapped_values = unique_values - mapped_values
        if unmapped_values:
            print(f"Unmapped categories in {var}: {unmapped_values}")


In [ ]:
# Apply the mappings to the ordinal variables
for var, mapping in ordinal_mappings.items():
    if var in train_features_dropped.columns:
        # Map the ordinal variable using the specified mapping
        train_features_dropped[var] = train_features_dropped[var].map(mapping)
    else:
        print(f"Warning: {var} not found in the dataframe columns.")

# Handle missing values if needed
# For example, fill NaN values with the median of the column
for var in ordinal_mappings.keys():
    if var in train_features_dropped.columns:
        median_value = train_features_dropped[var].median()
        train_features_dropped[var].fillna(median_value, inplace=True)

In [ ]:
# Check the unique values after encoding
for var in ordinal_mappings.keys():
    if var in train_features_dropped.columns:
        print(f"Unique values in {var} after encoding: {train_features_dropped[var].unique()}")


Unique values in age_03 after encoding: [1. 3. 2. 0. 4.]
Unique values in age_12 after encoding: [2. 1. 4. 3. 0.]
Unique values in edu_gru_03 after encoding: [1. 3. 0. 2. 4.]
Unique values in edu_gru_12 after encoding: [0. 3. 1. 2. 4.]
Unique values in n_living_child_03 after encoding: [3. 1. 0. 2. 4.]
Unique values in n_living_child_12 after encoding: [1. 3. 0. 2. 4.]
Unique values in glob_hlth_03 after encoding: [2. 1. 3. 5. 4.]
Unique values in glob_hlth_12 after encoding: [2. 3. 4. 1. 5.]
Unique values in bmi_12 after encoding: [3. 4. 2. 1. 5.]
Unique values in decis_famil_12 after encoding: [2. 3. 1.]
Unique values in decis_personal_12 after encoding: [3 2 1]
Unique values in satis_ideal_12 after encoding: [1. 3. 2.]
Unique values in satis_excel_12 after encoding: [1. 2. 3.]
Unique values in satis_fine_12 after encoding: [3. 1. 2.]
Unique values in cosas_imp_12 after encoding: [3. 2. 1.]
Unique values in wouldnt_change_12 after encoding: [1. 3. 2.]
Unique values in memory_12 after

In [ ]:
# Checking data types for each column in train_features to identify potential mismatches
train_features_dropped.dtypes


uid                         object
age_03                     float64
urban_03                    object
married_03                  object
n_mar_03                   float64
                            ...   
hinc_cap_03_missing          int64
rinc_pension_03_missing      int64
sinc_pension_03_missing      int64
searnings_12_missing         int64
sinc_pension_12_missing      int64
Length: 233, dtype: object

1. Identifiers:

    - uid is appropriately set as an object type, as it uniquely identifies each individual.
2. Categorical Variables:

    - Columns like age_03, urban_03, married_03, and other demographic and descriptive variables are currently object types. These will need encoding for use in machine learning models.
    - Categorical columns include some with ordinal information (e.g., age ranges, education levels) that might benefit from ordinal encoding to retain order.
3. Numerical Variables:

    - Variables related to counts (e.g., n_mar_03, n_adl_03, n_iadl_03) and binary indicators (0 or 1) are appropriately represented as float64.
    - Economic variables like hincome_12, hinc_business_12, and similar income-based columns are correctly identified as float64 but may need transformations due to skewness.


In [ ]:
nominal_variables

['uid',
 'urban_03',
 'married_03',
 'employment_03',
 'urban_12',
 'married_12',
 'employment_12',
 'ragender',
 'sgender_03',
 'sgender_12',
 'rjlocc_m_12',
 'j11_12']

For the nominal variables, we can use pandas' get_dummies() function to perform one-hot encoding.

In [ ]:
# Perform one-hot encoding on the nominal variables
train_encoded = pd.get_dummies(train_features_dropped, columns=nominal_variables, drop_first=False)

# Checking the number of columns after one-hot encoding
print(f"Number of columns after one-hot encoding: {train_encoded.shape[1]}")


Number of columns after one-hot encoding: 3550


In [ ]:
# Identifying categorical variables by checking columns with 'object' data type in train_features
categorical_columns = train_features.select_dtypes(include=['object']).columns.tolist()
categorical_columns


['uid',
 'age_03',
 'urban_03',
 'married_03',
 'edu_gru_03',
 'n_living_child_03',
 'glob_hlth_03',
 'bmi_03',
 'decis_famil_03',
 'employment_03',
 'age_12',
 'urban_12',
 'married_12',
 'edu_gru_12',
 'n_living_child_12',
 'glob_hlth_12',
 'bmi_12',
 'decis_famil_12',
 'decis_personal_12',
 'employment_12',
 'satis_ideal_12',
 'satis_excel_12',
 'satis_fine_12',
 'cosas_imp_12',
 'wouldnt_change_12',
 'memory_12',
 'ragender',
 'rameduc_m',
 'rafeduc_m',
 'sgender_03',
 'rjlocc_m_03',
 'rjobend_reason_03',
 'rrelgimp_03',
 'sgender_12',
 'rjlocc_m_12',
 'rjobend_reason_12',
 'rrelgimp_12',
 'rrfcntx_m_12',
 'rsocact_m_12',
 'rrelgwk_12',
 'a22_12',
 'a33b_12',
 'a34_12',
 'j11_12']

The dataset includes the following categorical variables:

1. Demographic and Background Information:

    - age_03, urban_03, married_03, edu_gru_03, n_living_child_03, glob_hlth_03, bmi_03, decis_famil_03, employment_03 (and their 2012 counterparts).
    - Static demographics: ragender (gender), rameduc_m (mother’s education), and rafeduc_m (father’s education).
2. Health and Well-being:

    - Self-reported variables like satis_ideal_12, satis_excel_12, memory_12, and glob_hlth_12 reflect self-perceived health or life satisfaction.
    - Variables related to body mass (bmi_03, bmi_12), activity, and health coverage.
3. Employment and Socioeconomic Information:

    - Employment specifics such as rjlocc_m_03 (job category), rjobend_reason_03 (reason for job end), rrelgimp_03 (religion importance), and similar 2012 counterparts.
4. Lifestyle and Social Interaction:

    - Variables indicating social interactions (rrfcntx_m_12, rsocact_m_12) and household decision-making (decis_famil_12, decis_personal_12).
5. Migration and Residency:

    - U.S. migration and residency status (a22_12, a33b_12), including whether individuals worked or lived in the U.S. and English proficiency (a34_12).

These variables will require encoding for modeling, with particular attention to ordinal categories where ordering matters

In [ ]:
# Creating Python lists for ordinal and nominal variables based on previous identification

# List of ordinal variables
ordinal_variables = [
    'age_03', 'age_12', 'edu_gru_03', 'edu_gru_12', 'glob_hlth_03', 'glob_hlth_12', 
    'bmi_03', 'bmi_12', 'n_depr_03', 'n_depr_12', 'satis_ideal_12', 'satis_excel_12', 
    'satis_fine_12', 'cosas_imp_12', 'wouldnt_change_12', 'decis_famil_03', 
    'decis_famil_12', 'decis_personal_12', 'rrelgimp_03', 'rrelgimp_12', 'rrfcntx_m_12', 
    'rsocact_m_12', 'memory_12'
]

# List of nominal variables
nominal_variables = [
    'uid', 'married_03', 'married_12', 'urban_03', 'urban_12', 'n_living_child_03', 
    'n_living_child_12', 'employment_03', 'employment_12', 'ragender', 'sgender_03', 
    'sgender_12', 'rameduc_m', 'rafeduc_m', 'rjlocc_m_03', 'rjobend_reason_03', 
    'rjlocc_m_12', 'rjobend_reason_12', 'imss_03', 'issste_03', 'pem_def_mar_03', 
    'a22_12', 'a33b_12', 'a34_12', 'j11_12'
]

# Display the lists to confirm creation
ordinal_variables, nominal_variables


(['age_03',
  'age_12',
  'edu_gru_03',
  'edu_gru_12',
  'glob_hlth_03',
  'glob_hlth_12',
  'bmi_03',
  'bmi_12',
  'n_depr_03',
  'n_depr_12',
  'satis_ideal_12',
  'satis_excel_12',
  'satis_fine_12',
  'cosas_imp_12',
  'wouldnt_change_12',
  'decis_famil_03',
  'decis_famil_12',
  'decis_personal_12',
  'rrelgimp_03',
  'rrelgimp_12',
  'rrfcntx_m_12',
  'rsocact_m_12',
  'memory_12'],
 ['uid',
  'married_03',
  'married_12',
  'urban_03',
  'urban_12',
  'n_living_child_03',
  'n_living_child_12',
  'employment_03',
  'employment_12',
  'ragender',
  'sgender_03',
  'sgender_12',
  'rameduc_m',
  'rafeduc_m',
  'rjlocc_m_03',
  'rjobend_reason_03',
  'rjlocc_m_12',
  'rjobend_reason_12',
  'imss_03',
  'issste_03',
  'pem_def_mar_03',
  'a22_12',
  'a33b_12',
  'a34_12',
  'j11_12'])